# Reforma

## Preâmbulo

In [1]:
import numpy as np
import pandas as pd
import psycopg2
import pandas.io.sql as sqlio
import time

# Track execution time                                                          
start_time = time.time()

### Parâmetros

In [2]:
# Simulation Parameters
#DBTABLE = 'FATO_AUXILIO_SAMPLE'
DBTABLE = 'FATO_AUXILIO'
ANO_INICIO = 1995
ANO_FIM = 2016
DADOS_FAZENDA = '../dataset/dados_fazenda.xlsx'

# Conection parameters
HOST='tama'
PORT='5432'
DBNAME='prevdb'
USER='prevdb_user'
PASS='pr3v'

### Biblioteca

In [3]:
def ds_query(sql_query):
    """
        Query Dataset

    Parâmetros
    ----------
        sql : string
            SQL query to be performed against the dataset
        
    Retorno
    -------
        Pandas Dataframe
    """
    # Connect to an existing database
    try:
        conn = psycopg2.connect("host='{}' port={} dbname='{}'user={} password={}"
                .format(HOST, PORT, DBNAME, USER, PASS))
        df = sqlio.read_sql_query(sql, conn)
        # Close communication with the database                                     
        conn.close()
        return df
    except:
        print("Unable to connect to the database")
        return

## Simulação

### Aposentados RGPS em ANO_FIM

In [9]:
sql = """
SELECT 
	ESPECIE
	,DIB
	,CLIENTELA
	,SEXO
	,SITUACAO
	,TEMPO_CONTRIB
	,IDADE_DIB
	,VL_RMI
FROM {table_name}
WHERE DIB > {ano}*10000 AND DIB < ({ano}+1)*10000
    AND ESPECIE IN (41 , 42)
    AND SEXO IN (3, 1)
""".format(table_name=DBTABLE,
           ano=ANO_FIM)
df = ds_query(sql)

In [13]:
print('ANO FIM = {}'.format(ANO_FIM))
out_df = df[['idade_dib','sexo']].groupby('idade_dib').count()
out_df

ANO FIM = 2016


KeyError: "['idade_dib' 'sexo'] not in index"

### Aposentados PEC 6/2019 em ANO_FIM

In [6]:
sql = """
SELECT 
	ESPECIE
	,SEXO
	,PEC6_IDADE_DIB
	--,PEC6_TEMPO_CONTRIB
	,PEC6_GAP
	,PEC6_ANO_DIB
	,PEC6_PROB
FROM FATO_REFORMA 
""".format(ano=ANO_FIM)
fato_pessoa = ds_query(sql)

In [11]:
print('ANO FIM = {}'.format(ANO_FIM))
df = fato_pessoa[fato_pessoa['pec6_ano_dib']==ANO_FIM][['pec6_idade_dib','pec6_prob']]
df = df.groupby('pec6_idade_dib').sum()
df['qtd'] = df['pec6_prob'].apply(lambda x: round(x))
df.to_csv('./2016_concessoes_pec.csv')
df

ANO FIM = 2016


,pec6_prob,qtd
pec6_idade_dib,,
60,309208.751763,309209
61,9169.738036,9170
62,9989.425213,9989
63,10547.409563,10547
64,11851.422126,11851
65,19574.402568,19574
66,20031.467835,20031
67,15533.559553,15534
68,13850.209011,13850


## Considerações

* **Probabilidade de a pessoa não querer se aposentar na idade mínima**
 * Utilizar probabilidades da base?
 * Como mensurar a "vontade de trabalhar mais para aumentar o valor do benefício"?
* **Aposentadoria por idade de homens**
 * Não sofre alterações em termos de quantidade na PEC 6/2019?
* **Tempo de Contribuição**
 * É válido condisiderar que o ano de inicio de contribuição como:  `ano_inicio_contrib = ano_dib - tempo_contrib`
 * Essa abordagem desconsidera períodos de desemprego/i

## Cleanup

In [8]:
# Print out elapsed time                                                        
elapsed_time = (time.time() - start_time) / 60                                  
print("\nExecution time: {0:0.4f} minutes.".format(elapsed_time))


Execution time: 1.2057 minutes.
